# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Saldanha,0,ZA,1579902556,82,-33.01,17.94,66.20,5.82
1,Itarema,41,BR,1579902556,68,-2.92,-39.92,81.05,7.99
2,Bitung,20,ID,1579902556,94,1.45,125.18,73.40,0.63
3,Laguna,1,US,1579902556,82,38.42,-121.42,66.99,4.70
4,Mataura,98,NZ,1579902556,95,-46.19,168.86,59.00,10.00
...,...,...,...,...,...,...,...,...,...
570,Benguela,85,AO,1579902631,79,-12.58,13.41,78.35,4.14
571,Sikasso,72,ML,1579902631,10,10.92,-7.00,78.53,6.29
572,Rodrigues Alves,20,BR,1579902631,62,-7.74,-72.65,87.80,3.36
573,Ambon,20,ID,1579902632,88,-3.70,128.18,75.20,9.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [11]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
135,Port Hedland,0,AU,1579902574,94,-20.32,118.57,78.80,2.24
207,Necochea,0,AR,1579902583,85,-38.55,-58.74,71.01,7.00
263,Balabac,0,PH,1579902590,82,7.99,117.06,79.52,7.47
357,Faxinal,0,BR,1579902603,70,-24.00,-51.32,79.20,4.36
373,San Vicente,0,AR,1579902605,65,-26.62,-54.13,79.00,6.31
398,Cockburn Town,0,TC,1579902609,53,21.46,-71.14,76.26,3.80


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
135,Port Hedland,AU,-20.32,118.57,
207,Necochea,AR,-38.55,-58.74,
263,Balabac,PH,7.99,117.06,
357,Faxinal,BR,-24.00,-51.32,
373,San Vicente,AR,-26.62,-54.13,
398,Cockburn Town,TC,21.46,-71.14,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 135: Port Hedland.
Closest hotel in Port Hedland is The Esplanade Hotel.
------------
Retrieving Results for Index 207: Necochea.
Closest hotel in Necochea is Hotel Real.
------------
Retrieving Results for Index 263: Balabac.
Closest hotel in Balabac is Sing and Swing Lodge.
------------
Retrieving Results for Index 357: Faxinal.
Closest hotel in Faxinal is Hotel Shalom.
------------
Retrieving Results for Index 373: San Vicente.
Closest hotel in San Vicente is Hospedaje La Luz.
------------
Retrieving Results for Index 398: Cockburn Town.
Closest hotel in Cockburn Town is Bohio Dive Resort.
------------
-------End of Search-------


In [9]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
135,Port Hedland,AU,-20.32,118.57,The Esplanade Hotel
207,Necochea,AR,-38.55,-58.74,Hotel Real
263,Balabac,PH,7.99,117.06,Sing and Swing Lodge
357,Faxinal,BR,-24.00,-51.32,Hotel Shalom
373,San Vicente,AR,-26.62,-54.13,Hospedaje La Luz
398,Cockburn Town,TC,21.46,-71.14,Bohio Dive Resort


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))